In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spy
from tqdm import tqdm
import time
import pickle


# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_rows',200)
pd.set_option('display.max_colwidth',None)

from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# icd9 or icd10 (seprerately)

In [2]:
ICD_VER=9

# load data_all

In [3]:
with open('./results/data_cl1_icd{}_all.pickle'.format(ICD_VER), 'rb') as f:
    cl1_dat=pickle.load(f)
    
with open('./results/data_cl0_icd{}_all.pickle'.format(ICD_VER), 'rb') as f:
    cl0_dat=pickle.load(f)

len(set(cl1_dat.columns)|set(cl0_dat.columns))
cl1_dat
cl0_dat

5879

,subject_id,gender,H_time,H_age,00845,0088,0090,0310,0312,0340,...,V854,V8541,V8542,V8543,V8544,V860,V8741,V8801,V9010,HF_diff
0,10001877,M,2149-05-21 15:53:00,89,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,10029106,M,2188-08-21 12:55:00,87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10037483,F,2159-09-07 17:44:00,73,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,10038852,F,2151-01-06 15:28:00,93,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,10041599,M,2122-07-15 18:53:00,71,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,19948170,F,2177-12-18 23:04:00,74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1385,19975121,F,2117-03-08 18:50:00,87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1386,19978248,F,2148-05-20 18:12:00,93,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1387,19993776,M,2133-02-05 07:15:00,83,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


,subject_id,gender,H_time,H_age,0020,0030,00329,0038,0039,0040,...,V8745,V8801,V8802,V8811,V8812,V9011,V9081,V9089,V9103,HF_diff
0,10000635,F,2136-06-19 14:24:00,74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
1,10000764,M,2132-10-14 23:31:00,86,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
2,10001176,F,2186-11-29 03:56:00,64,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
3,10001217,F,2157-11-18 22:56:00,55,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
4,10001860,M,2188-03-27 01:54:00,84,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26695,19998350,M,2128-02-21 09:41:00,53,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
26696,19998497,F,2140-01-02 16:55:00,83,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
26697,19999068,M,2161-08-24 04:10:00,63,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail
26698,19999287,F,2197-07-26 03:29:00,77,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not_avail


# Remove icds not related to disease ans icds not in d_icd

In [4]:
not_related_disease_icd=['Y92230','Y92239','Y929','Z23','Z66','E8770','Z9981','Y831','R531','W1830XA',
                         'Z9181','W19XXXA','Z006','W010XXA','Y838','Z781','Y92009', 'Z79899','V4986',
                         '4414','V1581','E8889','E8859','V1588','V153','V4501','E8798','4168']
not_in_d_icd=['E780', 'I482', 'I272', 'M4806']
H_icd=['4019','I10']

not_used_cols=list(set(cl1_dat.columns)&set(not_in_d_icd+not_related_disease_icd+H_icd))
cl1_dat=cl1_dat.drop(columns=not_used_cols)

not_used_cols=list(set(cl0_dat.columns)&set(not_in_d_icd+not_related_disease_icd+H_icd))
cl0_dat=cl0_dat.drop(columns=not_used_cols)

# Generate and save data for each subgroup (AL, G1, G2, G3)
* select frequenly diagnosed icds only (>= 0.05)

In [5]:
def get_subgroup(df1, Hage_equalmore, Hage_less):
    df2=df1.loc[(Hage_equalmore<=df1['H_age'])&(df1['H_age']<Hage_less)].copy()
    return df2

In [6]:
ICD_VER
for Hage_th1, Hage_th2 in [(0,120),(0,65),(65,80),(80,120)]:
    (Hage_th1, Hage_th2)

    ## class1 dat proc
    # filtering in prev. icds that were freqeuntly diagnosed
    cl1_dat_gp=get_subgroup(cl1_dat, Hage_equalmore=Hage_th1, Hage_less=Hage_th2)
    cl1_prev_dis_df=cl1_dat_gp.iloc[:,4:-1]
    cl1_icd_perc=cl1_prev_dis_df.applymap(lambda x: x!=0).mean(axis=0)
    cl1_rare_icd=cl1_icd_perc.loc[cl1_icd_perc<0.05].index
    cl1_dat_gp_freq=cl1_dat_gp.drop(columns=cl1_rare_icd).copy()

    with open('./results/data_cl1_icd{}_{}_{}.pickle'.format(ICD_VER,Hage_th1, Hage_th2), 'wb') as f:
        pickle.dump(cl1_dat_gp_freq, f)
        print('cl1', cl1_dat_gp_freq.shape)

    ## class0 dat proc
    # filtering in prev. icds determined in cl1_dat_gp
    cl0_dat_gp=get_subgroup(cl0_dat, Hage_equalmore=Hage_th1, Hage_less=Hage_th2)
    cl0_dat_qp_freq=cl0_dat_gp[cl1_dat_gp_freq.columns].copy()

    with open('./results/data_cl0_icd{}_{}_{}.pickle'.format(ICD_VER,Hage_th1, Hage_th2), 'wb') as f:
        pickle.dump(cl0_dat_qp_freq, f)
        print('cl0', cl0_dat_qp_freq.shape)
        

9

(0, 120)

cl1 (1389, 41)
cl0 (26700, 41)


(0, 65)

cl1 (303, 39)
cl0 (12737, 39)


(65, 80)

cl1 (520, 40)
cl0 (8574, 40)


(80, 120)

cl1 (566, 41)
cl0 (5389, 41)
